In [1]:
import torch
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset
from transformers import  RobertaConfig, RobertaModel, RobertaTokenizer
import argparse
import json
import os
from model2 import Model
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset
import random
import multiprocessing
from tqdm import tqdm, trange
import numpy as np
import javalang
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
np.random.seed(0)
import seaborn as sns
import collections
import pickle
import sklearn
from matplotlib import cm
from sklearn import manifold

sns.set_style("darkgrid")
BLACK = "k"
GREEN = "#59d98e"
SEA = "#159d82"
BLUE = "#3498db"
PURPLE = "#9b59b6"
GREY = "#95a5a6"
RED = "#e74c3c"
ORANGE = "#f39c12"
YELLOW = "#f1c40f"
GRAY = "#95a5a6"
BROWN = "#8e44ad"
CYAN = "#8e44ad"

/Users/jirigesi/anaconda3/envs/CodeBert/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 input_tokens,
                 input_ids,
                 idx,
                 label,

    ):
        self.input_tokens = input_tokens
        self.input_ids = input_ids
        self.idx=str(idx)
        self.label=label

In [8]:
def convert_examples_to_features(js,tokenizer):
    #source
    code=' '.join(js['func'].split())
    code_tokens=tokenizer.tokenize(code)[:block_size-2]
    source_tokens =[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
    source_ids =  tokenizer.convert_tokens_to_ids(source_tokens)
    padding_length = block_size - len(source_ids)
    source_ids+=[tokenizer.pad_token_id]*padding_length
    
    return InputFeatures(source_tokens,source_ids,js['idx'],js['target'])

In [4]:
class TextDataset(Dataset):
    def __init__(self, tokenizer, file_path=None):
        self.examples = []
        with open(file_path) as f:
            for line in f:
                js=json.loads(line.strip())
                self.examples.append(convert_examples_to_features(js,tokenizer))
        if 'train' in file_path:
            for idx, example in enumerate(self.examples[:3]):
                    logger.info("*** Example ***")
                    logger.info("idx: {}".format(idx))
                    logger.info("label: {}".format(example.label))
                    logger.info("input_tokens: {}".format([x.replace('\u0120','_') for x in example.input_tokens]))
                    logger.info("input_ids: {}".format(' '.join(map(str, example.input_ids))))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):       
        return torch.tensor(self.examples[i].input_ids),torch.tensor(self.examples[i].label)
         

In [5]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYHTONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = RobertaConfig.from_pretrained('microsoft/codebert-base')

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

model = RobertaModel.from_pretrained('microsoft/codebert-base',
                                    output_attentions=True, 
                                    output_hidden_states=True)

model=Model(model,config,tokenizer)

checkpoint_prefix = "saved_models/CD4DD/model_CD.bin"

model.load_state_dict(torch.load(checkpoint_prefix, map_location=torch.device('cpu')))

model = model.to(device)

In [9]:
block_size = 400
test_data_file = "../dataset/test.jsonl"
# Loop to handle MNLI double evaluation (matched, mis-matched)
eval_dataset = TextDataset(tokenizer,test_data_file)

In [13]:
len(eval_dataset), len(eval_dataset[0])

(2732, 2)

In [57]:
data = []
with open(test_data_file) as f:
    for line in f:
        js=json.loads(line.strip())
        data.append((js['target'], ' '.join(js['func'].split())))

In [58]:
len(data)

2732

In [59]:
data[0]

(0,
 'int ff_get_wav_header(AVFormatContext *s, AVIOContext *pb, AVCodecContext *codec, int size, int big_endian) { int id; uint64_t bitrate; if (size < 14) { avpriv_request_sample(codec, "wav header size < 14"); return AVERROR_INVALIDDATA; } codec->codec_type = AVMEDIA_TYPE_AUDIO; if (!big_endian) { id = avio_rl16(pb); if (id != 0x0165) { codec->channels = avio_rl16(pb); codec->sample_rate = avio_rl32(pb); bitrate = avio_rl32(pb) * 8LL; codec->block_align = avio_rl16(pb); } } else { id = avio_rb16(pb); codec->channels = avio_rb16(pb); codec->sample_rate = avio_rb32(pb); bitrate = avio_rb32(pb) * 8LL; codec->block_align = avio_rb16(pb); } if (size == 14) { /* We\'re dealing with plain vanilla WAVEFORMAT */ codec->bits_per_coded_sample = 8; } else { if (!big_endian) { codec->bits_per_coded_sample = avio_rl16(pb); } else { codec->bits_per_coded_sample = avio_rb16(pb); } } if (id == 0xFFFE) { codec->codec_tag = 0; } else { codec->codec_tag = id; codec->codec_id = ff_wav_codec_get_id(id, c

In [29]:
def get_syntax_types_for_code(code_snippet):
  types = ["[CLS]"]
  code = ["<s>"]
  tree = list(javalang.tokenizer.tokenize(code_snippet))
  
  for i in tree:
    j = str(i)
    j = j.split(" ")
    if j[1] == '"MASK"':
      types.append('[MASK]')
      code.append('<mask>')
    else:
      types.append(j[0].lower())
      code.append(j[1][1:-1])
    
  types.append("[SEP]")
  code.append("</s>")
  return np.array(types), ' '.join(code)

In [31]:
code_sample = data[0]
types, rewrote_code = get_syntax_types_for_code(code_sample[1])

In [33]:
len(types), len(rewrote_code.split())

(636, 634)

In [34]:
def get_start_end_of_token_when_tokenized(code, types, tokenizer):
  reindexed_types = []
  start = 0
  end = 0
  for index, each_token in enumerate(code.split(" ")):
    tokenized_list = tokenizer.tokenize(each_token)
    for i in range(len(tokenized_list)):
      end += 1
    reindexed_types.append((start, end-1))
    start = end
  return reindexed_types

In [72]:
def getSyntaxAttentionScore(model, data, tokenizer, syntaxList):
    block_size = 400
    all_instances = []
    number = 0
    for code_sample in tqdm(data):
        Instantce_Result = {}
        try: 
            for syntaxType in syntaxList:
                Instantce_Result[syntaxType] = []


            types_1, rewrote_code_1 = get_syntax_types_for_code(code_sample[1])
            
            tokenized_ids_1 = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(rewrote_code_1))

            if len(tokenized_ids_1) > 400:
                tokenized_ids_1 = tokenized_ids_1[:399] + [tokenizer.sep_token_id]
            
            padding_length = block_size - len(tokenized_ids_1)
            tokenized_ids_1+=[tokenizer.pad_token_id]*padding_length
            labels = code_sample[0]

            source_ids = torch.tensor(tokenized_ids_1).unsqueeze(0).to(device)
            labels = torch.tensor(labels).unsqueeze(0).to(device)

            with torch.no_grad():
                output = model(source_ids,labels)

            _attention = output[2].attentions
            start_end = get_start_end_of_token_when_tokenized(rewrote_code_1, types_1, tokenizer)
            
            for syntaxType in syntaxList:
                attention_weights = [[[] for col in range(12)] for row in range(12)]
                for layer in range(12):
                    for head in range(12):
                        for each_sep_index in np.where(types_1==syntaxType)[0]:
                            start_index, end_index = start_end[each_sep_index]
                            interim_value = _attention[layer][0][head][:, start_index:end_index+1].mean().cpu().detach().numpy()
                            if np.isnan(interim_value):
                                pass
                            else: 
                                attention_weights[layer][head].append(interim_value)     
                if np.array(attention_weights).shape[2] != 0:
                    Instantce_Result[syntaxType].append(np.array(attention_weights))
            all_instances.append(Instantce_Result)
        except:
            print("error")
            
    return all_instances

In [73]:
syntax_list = ['annotation', 'basictype', 'boolean', 
          'decimalinteger', 'identifier', 'keyword',
          'modifier', 'operator', 'separator', 'null',
          'string', 'decimalfloatingpoint']

In [76]:
syntax_attention_weights = getSyntaxAttentionScore(model, data, tokenizer, syntax_list)

  0%|          | 4/2732 [00:07<1:32:12,  2.03s/it]

error


  2%|▏         | 44/2732 [00:38<36:28,  1.23it/s] 

error


  2%|▏         | 49/2732 [00:41<28:17,  1.58it/s]

...

  4%|▍         | 103/2732 [01:27<29:27,  1.49it/s]

error


  4%|▍         | 107/2732 [01:29<27:04,  1.62it/s]

error
error


  5%|▍         | 130/2732 [01:49<35:00,  1.24it/s]

In [75]:
len(syntax_attention_weights)

9